In [290]:
# Import Required Packages
import lxml
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import urllib
import urllib.request as urllib2
import os
import requests
import pandas as pd

In [291]:
# Get URL
url = 'https://bceagles.com/roster.aspx?roster=384&path='
# establish a session to request website
user_agent = {'User-agent': 'Mozilla/5.0'}
urlSplit = url.split('/')
# Create string to represent the base url
baseUrl = urlSplit[0] + '/' + urlSplit[1] + '/' + urlSplit[2]

In [292]:
# Check if we get a response
r = requests.get(url, headers = user_agent)
# Make the soup
soup = BeautifulSoup(r.text, 'html.parser')

In [297]:
# Create a list of the players on the roster.
players = list()
firstName = list()
lastName = list()
numbers = list()
bat = list()
throw = list()
pos = list()

# Scrape the players name
for item in soup.findAll('td', {"class": "sidearm-table-player-name"}):
    name = item.text
    players.append(item.text)
    if('  ' in name):
        fullName = name.split('  ')
    else: fullName = name.split(' ')
    firstName.append(fullName[0])
    lastName.append(fullName[1])
players

# Scrape te players jersey number
for item in soup.findAll('td', {"class": "roster_jerseynum"}):
    numbers.append(item.text)
    
    
# Scrape the players batting/throwing side    
for item in soup.findAll('td', {"class": 'sidearm-table-custom-field'}):
    b_t = item.text.split('/')
    if(b_t[0] == 'R'):
        bat.append('Right')
    else: bat.append('Left')
    if(len(b_t) == 2):
        if(b_t[1] == 'R'):
            throw.append('Right')
        else: throw.append('Left')
    else: throw.append('Right')
        
# Scrape the players position
for item in soup.findAll('td', {"class": "rp_position_short"}):
    pos.append(item.text)
    
    
#########################################################
# Get user input for Team Abbrieviation:
#########################################################
ta = 'BC'

#########################################################
# Get user input for Team Organization:
#########################################################
to = 'Boston'

In [298]:
# Construct the Data Frame and Display
frame = pd.DataFrame(numbers, columns=['Jersey'])
frame['First Name'] = firstName
frame['Last Name'] = lastName
frame['Pitching Strength'] = throw
frame['Hitting Strength'] = bat
frame['Position'] = pos
frame['Team Abbrieviation'] = ta
frame['Team Organization'] = to
frame

,Jersey,First Name,Last Name,Pitching Strength,Hitting Strength,Position,Team Abbrieviation,Team Organization
0,1,Susannah,Anderson,Right,Right,P/1B,BC,Boston
1,2,Lexi,DiEmmanuele,Right,Left,OF,BC,Boston
2,3,Kennedy,Labshere,Right,Right,1B/3B,BC,Boston
3,5,Kristin,Giery,Right,Right,P/1B,BC,Boston
4,9,CC,Cook,Right,Right,P/OF,BC,Boston
5,10,Gianna,Randazza,Right,Right,OF,BC,Boston
6,12,Kendra,Friedt,Right,Right,P,BC,Boston
7,13,Allyson,Moore,Left,Left,C/1B,BC,Boston
8,14,Jules,Trevino,Left,Left,1B,BC,Boston
9,15,Jenna,Ergle,Right,Right,3B,BC,Boston


In [279]:
# create a list of links to player pages and display
playerAnchor = list()
anchors = soup.select('td.sidearm-table-player-name a', href = True)
for a in anchors:
    playerAnchor.append(baseUrl + a['href'])
playerAnchor

['https://gocards.com/sports/softball/roster/maddy-newman/8806',
 'https://gocards.com/sports/softball/roster/charley-butler/9044',
 'https://gocards.com/sports/softball/roster/blaire-bass/8797',
 'https://gocards.com/sports/softball/roster/riley-schindler/8811',
 'https://gocards.com/sports/softball/roster/erin-wilbur/9045',
 'https://gocards.com/sports/softball/roster/paige-schindler/8810',
 'https://gocards.com/sports/softball/roster/taylor-roby/8809',
 'https://gocards.com/sports/softball/roster/sidney-melton/8805',
 'https://gocards.com/sports/softball/roster/megan-hensley/8803',
 'https://gocards.com/sports/softball/roster/danielle-watson/8815',
 'https://gocards.com/sports/softball/roster/caitlin-ferguson/8800',
 'https://gocards.com/sports/softball/roster/darrianne-hale/8802',
 'https://gocards.com/sports/softball/roster/cassady-greenwood/9046',
 'https://gocards.com/sports/softball/roster/madison-ciaccio/8798',
 'https://gocards.com/sports/softball/roster/eden-mayo/9047',
 'ht

In [280]:
#########################################################
# User defines the path of where they save the file in Path
#########################################################

Path = './louisville/'

# find players and download images.
for idx, playerLink in enumerate(playerAnchor):
    # reference players link and create soup
    r = requests.get(playerLink, headers = user_agent)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # find images of the players
    player = soup.find('img', {'alt': players[idx]})
    print(player)
    # download the image
    if(player != None):
        imgurl = baseUrl + player['src'].split('?')[0]
        urllib2.urlretrieve(imgurl, os.path.join(Path, os.path.basename(imgurl)))

<img alt="Maddy Newman" src="/images/2019/8/29/Newman.jpg?width=300"/>
<img alt="Charley Butler" src="/images/2019/8/29/Butler.jpg?width=300"/>
None
None
None
<img alt="Paige  Schindler" src="/images/2019/8/29/Schindler.jpg?width=300"/>
<img alt="Taylor Roby" src="/images/2019/8/29/Roby.jpg?width=300"/>
None
None
<img alt="Danielle Watson" src="/images/2019/8/29/Watson.jpg?width=300"/>
<img alt="Caitlin Ferguson" src="/images/2019/8/29/Ferguson.jpg?width=300"/>
None
<img alt="Cassady Greenwood" src="/images/2019/8/29/Greenwood.jpg?width=300"/>
<img alt="Madison  Ciaccio" src="/images/2019/8/29/Ciaccio.jpg?width=300"/>
<img alt="Eden Mayo" src="/images/2019/8/29/Mayo.jpg?width=300"/>
<img alt="Jordyn Wolfe" src="/images/2019/8/29/Wolfe.jpg?width=300"/>
<img alt="Celene Funke" src="/images/2019/8/29/Funke.jpg?width=300"/>
<img alt="Rebecca Chung" src="/images/2019/8/29/Chung.jpg?width=300"/>
None
None


In [281]:
frame.to_csv(os.path.join(Path, 'roster.csv'))